# Artificial dataset generation
Since in this project I wasn't going to take thousands of images of the same playing cards, I only took one and the rest will be generated in this Jupyter notebook

In [1]:
import os
import numpy as np
import cv2
import random
from tqdm import tqdm

Iterate through every image, make copies of it with several deformations, and store its location in a train.yaml file for YOLO to train with. Original pictures are saved as validation.

In [2]:
def add_salt_and_pepper_noise(image, prob):
    noisy_image = np.copy(image)
    black = 0
    white = 255
    probs = np.random.rand(image.shape[0], image.shape[1])
    noisy_image[probs < (prob / 2)] = black
    noisy_image[probs > 1 - (prob / 2)] = white
    return noisy_image

def add_gaussian_noise(image, mean=0, var=0.01):
    row, col, ch = image.shape
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = image + gaussian * 255
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image.astype(np.uint8)

# https://stackoverflow.com/questions/43892506/opencv-python-rotate-image-without-cropping-sides 
def rotate_image(img, angle):
    size_reverse = np.array(img.shape[1::-1]) # swap x with y
    M = cv2.getRotationMatrix2D(tuple(size_reverse / 2.), angle, 1.)
    MM = np.absolute(M[:,:2])
    size_new = MM @ size_reverse
    M[:,-1] += (size_new - size_reverse) / 2.
    return cv2.warpAffine(img, M, tuple(size_new.astype(int)))

def apply_perspective_transform(image):
    h, w = image.shape[:2]
    pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    pts2 = np.float32([[random.randint(0, w//4), random.randint(0, h//4)], 
                       [w - random.randint(0, w//4), random.randint(0, h//4)], 
                       [random.randint(0, w//4), h - random.randint(0, h//4)], 
                       [w - random.randint(0, w//4), h - random.randint(0, h//4)]])
    
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

In [3]:

# This function is used to deform the image via rotation or perspective transform
def move_around(image):
    noisetype = random.randint(0, 1)
    if noisetype == 0:
        noisyImg = rotate_image(image, angle=random.uniform(-180, 180))
    else:
        noisyImg = apply_perspective_transform(image)
    return noisyImg

# This function is used to add noises to the image
def add_noises(image):
    noisetype = random.randint(0, 2)
    if noisetype == 0:
        noisyImg = add_salt_and_pepper_noise(image, prob=0.1)
    elif noisetype == 1:
        noisyImg = add_gaussian_noise(image, mean = 0, var = 0.1)
    else:
        noisyImg = image * random.random()*2.0
        noisyImg = np.clip(noisyImg, 0, 255)

    more_noise = random.randint(0, 1)
    if more_noise == 0:
        return add_noises(noisyImg)
    else:
        return noisyImg

Add names_dict to point at desired class to generate bbox

In [4]:
names_dict = {
    'cdg1': 0,
    'cdg2': 1,
    'cdg3': 2,
    'cdr1': 3,
    'cdr2': 4,
    'cdr3': 5,
    'cdv1': 6,
    'cdv2': 7,
    'cdv3': 8,
    'cgg1': 9,
    'cgg2': 10,
    'cgg3': 11,
    'cgr1': 12,
    'cgr2': 13,
    'cgr3': 14,
    'cgv1': 15,
    'cgv2': 16,
    'cgv3': 17,
    'clg1': 18,
    'clg2': 19,
    'clg3': 20,
    'clr1': 21,
    'clr2': 22,
    'clr3': 23,
    'clv1': 24,
    'clv2': 25,
    'clv3': 26,
    'rdg1': 27,
    'rdg2': 28,
    'rdg3': 29,
    'rdr1': 30,
    'rdr2': 31,
    'rdr3': 32,
    'rdv1': 33,
    'rdv2': 34,
    'rdv3': 35,
    'rgg1': 36,
    'rgg2': 37,
    'rgg3': 38,
    'rgr1': 39,
    'rgr2': 40,
    'rgr3': 41,
    'rgv1': 42,
    'rgv2': 43,
    'rgv3': 44,
    'rlg1': 45,
    'rlg2': 46,
    'rlg3': 47,
    'rlr1': 48,
    'rlr2': 49,
    'rlr3': 50,
    'rlv1': 51,
    'rlv2': 52,
    'rlv3': 53,
    'wdg1': 54,
    'wdg2': 55,
    'wdg3': 56,
    'wdr1': 57,
    'wdr2': 58,
    'wdr3': 59,
    'wdv1': 60,
    'wdv2': 61,
    'wdv3': 62,
    'wgg1': 63,
    'wgg2': 64,
    'wgg3': 65,
    'wgr1': 66,
    'wgr2': 67,
    'wgr3': 68,
    'wgv1': 69,
    'wgv2': 70,
    'wgv3': 71,
    'wlg1': 72,
    'wlg2': 73,
    'wlg3': 74,
    'wlr1': 75,
    'wlr2': 76,
    'wlr3': 77,
    'wlv1': 78,
    'wlv2': 79,
    'wlv3': 80
}

In [5]:
# Function to process images from Dataset folder and save to Segmented folder
def process_images(dataset_folder, output_folder, background_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Generate a list of background images
    background_images = []
    for root, _, filelist in os.walk(background_folder):
        for filename in filelist:
            # Check if the file is an image (you can add more formats if needed)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                img_path = os.path.join(root, filename)
                img_path = os.path.join(background_folder, img_path)
                image = cv2.imread(img_path)
                if image is not None:
                    height, width, channels = image.shape
                    if height > 100 and width > 100:
                        background_images.append(image)

    # Initialize the card number
    cardnum = 0
    card_duplicates = 10

    # Loop through all files in the dataset folder
    for filename in sorted(os.listdir(dataset_folder)):
        # Check if the file is an image (you can add more formats if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Read the image
            img_path = os.path.join(dataset_folder, filename)
            image = cv2.imread(img_path)

            if image is not None:
                print(f"Processing card {cardnum+1}/81...")
                for i in tqdm(range(int(card_duplicates))):
                    # Process the image to remove black background and trim
                    processed_image = move_around(image)
                    gray = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)
                    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
                    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


                    #Remove content outside the bounding box
                    cv2.drawContours(thresh, contours, -1, color=(255, 255, 255), thickness=cv2.FILLED)
                    processed_image = cv2.bitwise_and(processed_image, processed_image, mask=thresh)

                    # Place noisy image on a random background on a random position
                    background = random.choice(background_images)
                    background = cv2.resize(background, (640, 640))

                    rand_size = random.randint(100, 430)
                    resized_img = cv2.resize(processed_image, (rand_size, int(rand_size*1.3)))

                    #Find new bounding box
                    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
                    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    x, y, w, h = cv2.boundingRect(contours[0])
                    cv2.drawContours(thresh, contours, -1, color=(255, 255, 255), thickness=cv2.FILLED)

                    x_offset = random.randint(0, background.shape[1] - resized_img.shape[1])
                    y_offset = random.randint(0, background.shape[0] - resized_img.shape[0])
                                
                    mask = thresh != 0
                    background[y_offset:y_offset+resized_img.shape[0], x_offset:x_offset+resized_img.shape[1]][mask]= resized_img[mask]
                    processed_image = background

                    processed_image = add_noises(processed_image)
                    
                    # Save the processed image to the output folder
                    output_path = os.path.join(output_folder, str(cardnum)+".png")
                    cv2.imwrite(output_path, processed_image)

                    # Write bounding box in separate file
                    norm_x = (x + x_offset) / background.shape[1]
                    norm_y = (y + y_offset) / background.shape[0]
                    norm_w = w / background.shape[1]
                    norm_h = h / background.shape[0]
                    with open(os.path.join(output_folder, str(cardnum)+".txt"), 'w') as f:
                        f.write(f"{names_dict[filename.split('.')[0]]} {norm_x+norm_w/2} {norm_y+norm_h/2} {norm_w} {norm_h}\n")
                    
                    cardnum += 1

# Main execution
if __name__ == "__main__":
    dataset_folder = "Segmented"      # Folder containing the input images
    output_folder = "Validation_Dataset"     # Folder to save the processed images
    background_folder = "/media/jaume/1C8E63228E62F3A4/Users/Jaume/Documents/Youtube" # Folder containing the background images

    process_images(dataset_folder, output_folder, background_folder)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Processing card 1/81...


100%|██████████| 10/10 [00:01<00:00,  6.46it/s]


Processing card 11/81...


100%|██████████| 10/10 [00:00<00:00, 10.37it/s]


...

100%|██████████| 10/10 [00:00<00:00, 13.12it/s]


Processing card 791/81...


100%|██████████| 10/10 [00:01<00:00,  9.99it/s]


Processing card 801/81...


100%|██████████| 10/10 [00:01<00:00,  9.34it/s]
